In [1]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")


In [2]:
from utils.data_utils import load_movement_df, ward2size
from utils.plot_utils import *

path_to_data = os.path.join('..', '..', 'data')

# load scenarios for synthetic inferences
scenarios_df              = pd.read_csv(os.path.join(path_to_data, 'scenarios.csv'))
movement_df, ward2cluster = load_movement_df(path_to_data, True) # movement data
ward2size                 = ward2size(movement_df)


/var/folders/7p/jp6xqkvn5wb6ddl1fn0bhs980000gn/T/ipykernel_94414/1923229292.py:8: DtypeWarning: Columns (0,1,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  movement_df, ward2cluster = load_movement_df(path_to_data, True) # movement data


KeyboardInterrupt: 

In [ ]:
date_min         = pd.to_datetime("2020-02-01")
date_max         = pd.to_datetime("2021-02-28")
dates_simulation = pd.date_range(start=date_min, end=date_max)

γ_prior  = [0.01, 0.9]
β_prior  = [0.001, 0.05]

param_prior_dict      = {}
param_prior_dict["γ"] = γ_prior
param_prior_dict["β"] = β_prior

# Agent based model settings.
abm_settings                     = {}
abm_settings["num_patients"]     = movement_df.mrn_id.unique().shape[0]
abm_settings["num_wards"]        = movement_df.ward_id.unique().shape[0]
abm_settings["num_clusters"]     = len(set(list(ward2cluster.values())))
abm_settings["dates"]            = dates_simulation
abm_settings["num_ensembles"]    = 300

# Iterated filtering settings.
if2_settings                     = {}
if2_settings["num_params"]       = len(param_prior_dict)
if2_settings["num_observations"] = len(set(list(ward2cluster.values())))
if2_settings["lambda_inf"]       = 1.01        # Inflation for the EAKF.
if2_settings["num_iters_mif"]    = 20          # Number of iterations.
if2_settings["alpha_mif"]        = 0.8         # Variance shrinking factor.
if2_settings["type_cooling"]     = "geometric" # Type of cooling.
if2_settings["num_ensembles"]    = 300
if2_settings["oev_variance"]     = 0.2

## Simulate scenarios for synthetic inferences

In [ ]:
from utils.model_utils import simulate_model

for idx_row, row in scenarios_df.iterrows():

    θ = {}
    θ['γ'] = row['γ']
    θ['β'] = row['β']

    print("***-***-***-***-***-***")
    print("Simulating model...")
    ward_colonized, ward_nosocomial, ward_colonized_imported, ward_positive, ward_negative,\
    cluster_colonized, cluster_nosocomial, cluster_colonized_imported, cluster_positive, cluster_negative \
        = simulate_model(movement_df, ward2size, ward2cluster, θ, abm_settings)

    print("***-***-***-***-***-***")


    positive_sim   = ward_pos_sim.sum(-2)
    idx_use        = np.argsort(np.sum(positive_sim, 0))[abm_settings["num_ensembles"]//2]
    name_sims_save = f"simulation_infer.npz"

    np.savez_compressed(os.path.join('..', '..', 'results', 'synthetic_inferences', name_sims_save),
                                    positive                 = positive_sim,
                                    ward_colonized           = ward_colonized_sim,
                                    ward_nosocomial          = ward_nosocomial_sim,
                                    ward_colonized_imported  = ward_col_imp_sim,
                                    ward_positive            = ward_pos_sim,
                                    chunk_colonized          = chunk_colonized_sim,
                                    chunk_nosocomial         = chunk_nosocomial_sim,
                                    chunk_colonized_imported = chunk_col_imp_sim,
                                    chunk_positive           = chunk_pos_sim,
                                    idx_use                  = idx_use)



In [ ]:
#from utils_inference2 import compute_sen_oev, sample_params_uniform, sample_params_normal, cooling, random_walk_perturbation, inflate_ensembles, checkbound_params, eakf_step_multi_obs
def create_obs_infer(data_infer, abm_settings):
    # Create population level observations
    obs_pop_df                       = pd.DataFrame(columns=["date", "positives", "negatives"])
    obs_pop_df["date"]               = abm_settings["dates"]
    obs_pop_df["positives"]          = data_infer.pos_obs
    obs_pop_df["negatives"]          = data_infer.neg_obs

    # Resample every week
    obs_w_pop_df         = obs_pop_df.set_index("date").resample("W-Sun").sum()
    obs_w_pop_df["oev"]  = compute_sen_oev(obs_w_pop_df["positives"], var_obs=if2_settings["oev_variance"])
    obs_w_pop_df["oev"]  = np.minimum(obs_w_pop_df["oev"], 50*7)

    # Create ward chunks level observations
    obs_chunk_df         = pd.DataFrame(columns=["date"] + [f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])])
    obs_chunk_df["date"] = abm_settings["dates"]

    neg_chunk_df         = pd.DataFrame(columns=["date"] + [f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])])
    neg_chunk_df["date"] = abm_settings["dates"]

    for idx_chunk in range(abm_settings["num_clusters"]):
        obs_chunk_df[f"pos_{idx_chunk}"] = ward_pos_obs[:, idx_chunk]
        neg_chunk_df[f"pos_{idx_chunk}"] = ward_neg_obs[:, idx_chunk]

    # Resample every week
    obs_w_chunk_df         = obs_chunk_df.set_index("date").resample("W-Sun").sum()
    neg_w_chunk_df         = neg_chunk_df.set_index("date").resample("W-Sun").sum()

    for idx_chunk in range(abm_settings["num_clusters"]):
        obs_w_chunk_df[f"oev_{idx_chunk}"]  = compute_sen_oev(obs_w_chunk_df[f"pos_{idx_chunk}"] , var_obs=0.2)
        obs_w_chunk_df[f"oev_{idx_chunk}"]  = np.minimum(obs_w_chunk_df[f"oev_{idx_chunk}"], 50*7)
        neg_w_chunk_df[f"oev_{idx_chunk}"]  = compute_sen_oev(neg_w_chunk_df[f"pos_{idx_chunk}"] , var_obs=0.2)
        neg_w_chunk_df[f"oev_{idx_chunk}"]  = np.minimum(neg_w_chunk_df[f"oev_{idx_chunk}"], 50*7)

    return obs_w_chunk_df, neg_w_chunk_df


In [ ]:
def IF2_eakf_ABM(data_infer, model_infer, movement_df, param_prior, if2_settings, abm_settings, scenario_params, ward2community_dict, ward2size_dict, perturb_time=True, path_to_save=None, run_id=None, sequential=True):

    obs_w_chunk_df, neg_w_chunk_df = create_obs_infer(data_infer, abm_settings)

    cooling_factor   = cooling(if2_settings["num_iters_mif"], type_cool=if2_settings["type_cooling"], cooling_factor=if2_settings["alpha_mif"])

    param_range      = np.array([v for k, v in param_prior_dict.items()])
    std_param        = param_range[:,1] - param_range[:,0]
    SIG              = std_param ** 2 / 4; #  initial covariance of parameters

    # Perturbation is proportional to the prior range of search.
    perturbation     = np.array([std_param % list(np.round(std_param)+0.1)]).T
    num_steps        = len(obs_w_chunk_df)

    param_mean_iter  = np.full((if2_settings["num_params"],       if2_settings["num_iters_mif"]+1), np.nan)                                         # Array to store posterior parameters in iterations.
    para_post_all    = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array to store posterior parameters.
    param_iter       = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], if2_settings["num_iters_mif"]), np.nan)

    if sequential:
        obs_post_all_pos     = np.full((if2_settings["num_observations"], if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array for store posterior observations
        obs_post_all_neg     = np.full((if2_settings["num_observations"], if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array for store posterior observations

    else:
        obs_post_all   = np.full((if2_settings["num_observations"]*2, if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array for store posterior observations

    p_priors_all       = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], if2_settings["num_iters_mif"]), np.nan)

    dates_assimilation     = obs_w_chunk_df.index.get_level_values(0).values
    dates_assimilation[-1] = abm_settings["dates"][-1]

    alpha_ens    = np.random.uniform( 1/365, 1/175, size=(abm_settings["num_patients"], if2_settings["num_ensembles"]))
    perturb_time = True

    print(f"Running MIF  \n")
    for n in tqdm(range(if2_settings["num_iters_mif"])):
        if n==0: # Initial IF iteration

            p_prior               = sample_params_uniform(param_prior_dict, num_ensembles=if2_settings["num_ensembles"])
            param_mean_iter[:, n] = np.mean(p_prior, -1)
            p_priors_all[:,:,n]   = p_prior

        else:
            params_mean           = param_mean_iter[:,n]
            params_var            = SIG * cooling_factor[n]
            p_prior               = sample_params_normal(param_prior_dict, params_mean, params_var, num_ensembles=if2_settings["num_ensembles"])
            p_priors_all[:,:,n]   = p_prior

        patients_state    = np.zeros((abm_settings["num_patients"], if2_settings["num_ensembles"]))

        param_post_time   = np.full((if2_settings["num_params"], if2_settings["num_ensembles"], num_steps), np.nan)

        if sequential:
            obs_post_time_pos     = np.full((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"], num_steps), np.nan)
            obs_post_time_neg     = np.full((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"], num_steps), np.nan)

        else:
            obs_post_time     = np.full((abm_settings["num_wards_chunks"]*2, if2_settings["num_ensembles"], num_steps), np.nan)

        idx_date_update   = 0

        # Init observation arrays.
        chunk_pos_t = np.zeros((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"]))
        chunk_neg_t = np.zeros((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"]))

        for idx_date, date in enumerate(abm_settings["dates"]):
            #print(date)

            # Integrate model
            gamma_ens = p_prior[0, :]
            beta_ens  = p_prior[1, :]

            movement_date = movement_df.loc[date]
            patients_state, _, chunk_pos, _, chunk_neg = model_infer(patients_state, gamma_ens, beta_ens, alpha_ens, movement_date, ward_id2size, ward2community_dict)

            chunk_pos_t += chunk_pos
            chunk_neg_t += chunk_neg

            if pd.to_datetime(date) == pd.to_datetime(dates_assimilation[idx_date_update]):
                # Perturb parameters according to the define mapping
                if perturb_time:
                    # Transform parameters for perturbation
                    std_params = perturbation * cooling_factor[n]
                    p_prior    = random_walk_perturbation(p_prior, std_params, if2_settings["num_params"], if2_settings["num_ensembles"])

                # Inflate parameters
                p_prior = inflate_ensembles(p_prior, inflation_value=if2_settings["lambda_inf"], num_ensembles=if2_settings["num_ensembles"])
                p_prior = checkbound_params(param_prior_dict, p_prior, num_ensembles=if2_settings["num_ensembles"])

                # first adjust using only positives
                oev_pos   = obs_w_chunk_df.loc[date][[f"oev_{idx_chunk}" for idx_chunk in range(abm_settings["num_wards_chunks"])]].values
                pos_time  = obs_w_chunk_df.loc[date][[f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_wards_chunks"])]].values

                # then adjust using negatives
                oev_neg   = neg_w_chunk_df.loc[date][[f"oev_{idx_chunk}" for idx_chunk in range(abm_settings["num_wards_chunks"])]].values
                neg_time  = neg_w_chunk_df.loc[date][[f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_wards_chunks"])]].values

                param_post               = p_prior.copy()

                if sequential:

                    param_post, obs_post_pos = eakf_step_multi_obs(param_post, chunk_pos_t, np.expand_dims(pos_time, -1),  np.expand_dims(oev_pos, -1), param_prior_dict, int(if2_settings["num_observations"] )) # Use both positives to adjust
                    param_post               = checkbound_params(param_prior_dict, params_ens=param_post, num_ensembles=if2_settings["num_ensembles"])

                    param_post, obs_post_neg = eakf_step_multi_obs(param_post, chunk_neg_t, np.expand_dims(neg_time, -1),  np.expand_dims(oev_neg, -1), param_prior_dict, int(if2_settings["num_observations"] )) # Use negatives to adjust
                    param_post               = checkbound_params(param_prior_dict, params_ens=param_post, num_ensembles=if2_settings["num_ensembles"])
                    obs_post_time_pos[:,:,idx_date_update]    = obs_post_pos
                    obs_post_time_neg[:,:,idx_date_update]    = obs_post_neg

                else:
                    obs_ens_time             = np.concatenate([chunk_pos_t, chunk_neg_t])
                    obs_time                 = np.concatenate([pos_time, neg_time])
                    oev_time                 = np.concatenate([oev_pos, oev_neg])

                    param_post, obs_post     = eakf_step_multi_obs(param_post, obs_ens_time, np.expand_dims(obs_time, -1),  np.expand_dims(oev_time, -1), param_prior_dict, int(if2_settings["num_observations"]*2 )) # Use both positives and negatives in the inference.
                    param_post               = checkbound_params(param_prior_dict, params_ens=param_post, num_ensembles=if2_settings["num_ensembles"])
                    obs_post_time[:,:,idx_date_update]    = obs_post

                # Use posterior as next prior
                p_prior = param_post.copy()
                param_post_time[:,:,idx_date_update]  = param_post

                idx_date_update += 1

                chunk_pos_t = np.zeros((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"]))
                chunk_neg_t = np.zeros((abm_settings["num_wards_chunks"], if2_settings["num_ensembles"]))

        para_post_all[:,:,:,n]    = param_post_time
        param_mean_iter[:,n+1]    = param_post_time.mean(-1).mean(-1)

        obs_post_all_pos[:,:,:,n] = obs_post_time_pos
        obs_post_all_neg[:,:,:,n] = obs_post_time_neg

    return obs_w_chunk_df, neg_w_chunk_df, obs_post_all_pos, obs_post_all_neg, para_post_all, param_iter, param_mean_iter